<a href="https://colab.research.google.com/github/maggiemcc02/Adaptive_Meshing/blob/main/Newton_Meshes/Discretization_MeshSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
To create my adaptive meshing code, I will couple together a physical ODE solver and a mesh solver. The physical solver solves the problem of interest. It then gives the mesh solver the final approximations and the mesh it used to find these approximations. The mesh solver takes these two vectors and solves another ODE whose solution is the new and improved non-uniform mesh. The mesh solver gives the physical solver this new mesh and the whole process repeats.

In this notebook, my focus is on the mesh solver. In particular, I will show the steps taken to discretize the ODE that is solved in the mesh solver. Once I have been able to discretize the ODE, I will be able to define my system that I will need to solve in order to solve the ODE. To solve this system I need the jacobian. Therefore, I will also derive the jacobian from my system in this notebook.

### By: Maggie McCarthy, June 2022

## Discretizing the Mesh ODE

In the mesh solver, we are solving an ODE of the form:
$$ ( M(x), x_{\xi} )_{\xi} = 0$$
where $x$ is our solution function (our non-uniform mesh) that we are approximating on the uniform grid $\xi.$

To understand this discretization, we shall break it down piece by piece.

### The Indices
In the equations to follow you will encounter a subscript $i,$ and superscript $(0),$ and another superscript $(k).$ Before diving into these equations, I will clarify what these indices refer to.

The subscript $i$ refers to the $ith$ entry in a vector. Since we are working with a system of equations, the index $i$ refers to the $ith$ equation in our system which corresponds to the $ith$ row of the matrix and the $ith$ entry in a vector. 

The superscript $(k)$ is the Newton Method iteration counter. For each iteration of Newton's Method in our mesh solver, we are approximating and updating the values in a vector $x.$ Therefore, the vector $x$ has differant values for each iteration. To clarify which vector $x$ we are refering to, we use a superscript $(k)$ which counts the number of iterations of newtons method. 

The superscript $(0)$ refers to a vector inputted into our mesh solver. In our approximations of the mesh density function, we require the approximations of the solution to the physical problem (original problem we solve in the physical problem). We call this vector of approximations $U^{(0)}$ in our equations because it remains constant throughout the implementation of Newton's Method. 

For example, $x^{(k)}_i$ refers to the $ith$ entry in the vector $x$ used in the $kth$ Newton iteration.

### The Mesh Density Function $M.$
Our ODE has a function call to our mesh density function $M.$ Therefore, in each Newton Iteration we will need to call on this function to create our system and it's jacobian. As you will see, we actually call $M$ at the halfway points between our solution approximations, $x_i$ (half way between the non-unform grid points we are approximating). Therefore, I thought it best to define the equation for $M$ at $x_{i+\frac{1}{2}}$ and at $x_{i-\frac{1}{2}}$ so that we can use these equations as needed in our discretization. 

$$ M(x_{i+\frac{1}{2}}) = \sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2} $$
and 
$$ M(x_{i-\frac{1}{2}}) = \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2}.$$

### The Discretization
The method used to discretize this ODE can be found in Example 2.1 in Section 2.15 in Leveque's text. I will be breaking it down here step by step in relation to our problem.

1. Approximate $ M(x)x_{\xi} $ at points halfway between grid points. Remember that the grid points for $M$ come from the non-uniform grid $x.$ Therefore, we are approximating $M(x_{i+\frac{1}{2}})$ and $M(x_{i-\frac{1}{2}}).$ The grid points for $x$ come from the uniform grid $\xi$ where the mesh width is $\frac{1}{N}$ ($N$ is the number of grid points). Therefore, we are approximating $X_{i+\frac{1}{2}} \approx x(\xi_{i + \frac{1}{2}})$ and $X_{i-\frac{1}{2}} \approx x(\xi_{i - \frac{1}{2}}).$ The equations for these approximations are:



$$M(x_{i+\frac{1}{2}})(X_{i+\frac{1}{2}}) = \sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2}\cdot \left( \frac{ x^{(k)}_{i+1} - x^{(k)}_i }{ \frac{1}{N}} \right)$$
and 
$$M(x_{i-\frac{1}{2}})(X_{i-\frac{1}{2}}) = \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2}\cdot \left( \frac{ x^{(k)}_{i} - x^{(k)}_{i-1} }{ \frac{1}{N}} \right)$$

2. To approximate the whole ODE, we take the differance of these approximations to obtain a centered approximation:

$$ ( M(x), x_{\xi} )_{\xi} = \frac{1}{N} \left[\sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2}\cdot \left( \frac{ x^{(k)}_{i+1} - x^{(k)}_i }{ \frac{1}{N}} \right) - \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2}\cdot \left( \frac{ x^{(k)}_{i} - x^{(k)}_{i-1} }{ \frac{1}{N}} \right) \right]$$

3. We can cancel $\frac{1}{N}$ to obtain

$$ ( M(x), x_{\xi} )_{\xi} = \left[\sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2}\cdot \left( x^{(k)}_{i+1} - x^{(k)}_i  \right) - \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2}\cdot \left(x^{(k)}_{i} - x^{(k)}_{i-1} \right) \right]$$

4. Now we can expand multiply out this equation and collect like terms to get our discretization:

$$ ( M(x), x_{\xi} )_{\xi} = (x^{(k)}_{i-1}) \cdot \left( \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2} \right) - (x^{(k)}_{i}) \cdot \left( \sqrt{ 1 + \left( \frac{ U^{(0)}_{i} - U^{(0)}_{i-1}}{x^{(k)}_{i} - x^{(k)}_{i-1}} \right)^2} + \sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2} \right) + (x^{(k)}_{i+1}) \cdot \left(\sqrt{ 1 + \left( \frac{ U^{(0)}_{i+1} - U^{(0)}_{i}}{x^{(k)}_{i+1} - x^{(k)}_{i}} \right)^2} \right). $$

Using this discretization, we can define our system of $i$ equations and $i$ unknowns. 













